In [6]:
import pandas as pd

LOAD_PATH = '../../data/squadv2/base_dataset.csv'

In [7]:
df = pd.read_csv(LOAD_PATH, sep=';')

In [5]:
eval_df

,id,title,context,question,answers
0,56ddde6b9a695914005b9628,Normans,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,"{'text': ['France', 'France', 'France', 'Franc..."
1,56ddde6b9a695914005b9629,Normans,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,"{'text': ['10th and 11th centuries', 'in the 1..."
2,56ddde6b9a695914005b962a,Normans,The Normans (Norman: Nourmands; French: Norman...,From which countries did the Norse originate?,"{'text': ['Denmark, Iceland and Norway', 'Denm..."
3,56ddde6b9a695914005b962b,Normans,The Normans (Norman: Nourmands; French: Norman...,Who was the Norse leader?,"{'text': ['Rollo', 'Rollo', 'Rollo', 'Rollo'],..."
4,56ddde6b9a695914005b962c,Normans,The Normans (Norman: Nourmands; French: Norman...,What century did the Normans first gain their ...,"{'text': ['10th century', 'the first half of t..."
...,...,...,...,...,...
11868,5737aafd1c456719005744ff,Force,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,"{'text': ['sthène', 'sthène', 'sthène', 'sthèn..."
11869,5ad28ad0d7d075001a4299cc,Force,"The pound-force has a metric counterpart, less...",What does not have a metric counterpart?,"{'text': [], 'answer_start': []}"
11870,5ad28ad0d7d075001a4299cd,Force,"The pound-force has a metric counterpart, less...",What is the force exerted by standard gravity ...,"{'text': [], 'answer_start': []}"
11871,5ad28ad0d7d075001a4299ce,Force,"The pound-force has a metric counterpart, less...",What force leads to a commonly used unit of mass?,"{'text': [], 'answer_start': []}"
